In [1]:
import folium
import pickle
import pandas as pd
import networkx as nx

# Graph Analysis

Now that the graph is partially constructed, let's perform some analysis on it.

In [2]:
# Start by loading pickled data
G = pickle.load(open('data/processed/railgraph.pickle', 'rb'))
stop_id_to_name = pickle.load(open('data/processed/stop_id_to_name', 'rb'))
trips_grouped = pd.read_pickle('data/processed/trips_grouped.pickle')
railway_stops = pd.read_pickle('data/processed/railway_stops.pickle')

## Betweenness Centrality

*TODO: add short description about centrality measure and how it can be interpreted* 

In [3]:
# Compute betweenness centrality of graph (unweighted)
btwn_centrality = nx.betweenness_centrality(G)

In [4]:
# Take top 20
btwn_centrality_20 = dict(sorted(btwn_centrality.items(), key=lambda item: - item[1])[:20]).keys()

print('Top 20 stops with the highest betweenness centrality:')
for stop in btwn_centrality_20:
    print('{:.<20} {:.4f}'.format(stop_id_to_name[stop], btwn_centrality[stop]))

Top 20 stops with the highest betweenness centrality:
Zürich HB........... 0.4424
Bern................ 0.3523
Olten............... 0.2309
Lausanne............ 0.2017
Fribourg/Freiburg... 0.1965
Winterthur.......... 0.1902
Landquart........... 0.1146
St. Gallen.......... 0.1071
Spiez............... 0.1000
Montreux............ 0.0919
Thun................ 0.0917
Chur................ 0.0859
Luzern.............. 0.0763
Biel/Bienne......... 0.0744
Neuchâtel........... 0.0662
Aigle............... 0.0605
Visp................ 0.0582
Arth-Goldau......... 0.0570
Yverdon-les-Bains... 0.0566
Ins................. 0.0528


We can see that the 20 most central nodes according to the betweenness centrality are mostly Swiss agglomerations. However, we see some less major cities with a somewhat important centrality: i.e. Visp, Landquart, Ins. These are probably smalles cities through which has to pass because of their geographic localisation: i.e. the stop might be on the intersection of multiple lines for example. 

In [5]:
for node in G.nodes:
    G.nodes[node]['centrality'] = btwn_centrality[node]

In [ ]:
# Create map to visualize graph
m = folium.Map(location=[46.771413, 8.471689], zoom_start = 8, tiles='CartoDB Positron', height = '80%')

# Draw nodes
for node in G.nodes():
    lat, lon = G.nodes[node]['lat'], G.nodes[node]['lon']
    folium.CircleMarker(
        location = [lat, lon],
        popup = '{}, {}'.format(G.nodes[node]['name'],  G.nodes[node]['centrality']), 
        radius = 4 if node in btwn_centrality_20 else 1,
        color = 'green' if node in btwn_centrality_20 else '#3388ff',
        opacity = 1 if node in btwn_centrality_20 else 0.3,
        fill = True
    ).add_to(m)

m.save("network_betweenness.html")
m